# Laboratorio 04 | The Games of Nodes Part II

Agregaciones en Neo4j (Cypher) usando Count, Collect y Size

## Configuración del entorno

In [42]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os
import pandas as pd

In [43]:
load_dotenv()

URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))

driver = GraphDatabase.driver(URI, auth=AUTH)

def close_connection():
    driver.close()

def verify_connection():
    try:
        driver.verify_connectivity()
        print("Connection verified")
        with driver.session() as session:
            result = session.run("RETURN 1 AS value")
            value = result.single()["value"]
            print("Connection successful. Result:", value)
    except Exception as e:
        print("Connection failed:", e)

### Verificar la conexión

In [44]:
verify_connection()
# close_connection()

Connection verified
Connection successful. Result: 1


## **Acto 1: El Descenso de los Cineastas**

### **Investigación 1:** Determinar la cantidad de cineastas antes y después de 2010
- **Objetivo:** Determinar cuántos cineastas han ingresado a la industria antes y después del 2010.

In [45]:
def get_arround_2010():
  query1 = """
    MATCH (p:Person)
    WHERE p.yearJoined < 2010
    RETURN COUNT(p) AS CineastasAntes
  """
  query2 = """
    MATCH (p:Person)
    WHERE p.yearJoined >= 2010
    RETURN COUNT(p) AS CineastasDespues
  """
  with driver.session() as session:
    result1 = session.run(query1)
    cineastas_antes = result1.single()["CineastasAntes"]
    
    result2 = session.run(query2)
    cineastas_despues = result2.single()["CineastasDespues"]
    
    return cineastas_antes, cineastas_despues

cineastas_antes_2010, cineastas_despues_2010 = get_arround_2010()
print("Cineastas antes de 2010:", cineastas_antes_2010)
print("Cineastas después de 2010:", cineastas_despues_2010)

Cineastas antes de 2010: 6
Cineastas después de 2010: 4


### **Investigación 2:** Analizar los cambios en la industria con base en los géneros
- **Objetivo:** Investigar los cambios en la industria que podrían estar relacionados con los géneros de las películas. Uso de COUNT().

In [46]:
def get_movie_genres_count():
  query = """
  MATCH (m:Movie)-[:BELONGS_TO]->(g:Genre)
  RETURN g.name AS Genero, COUNT(m) AS NumPeliculas
  ORDER BY NumPeliculas DESC
  """
  with driver.session() as session:
    result = session.run(query)
    genres_count = [{"Genero": record["Genero"], "NumPeliculas": record["NumPeliculas"]} for record in result]
    return genres_count

movie_genres_count = get_movie_genres_count()

df_movie_genres_count = pd.DataFrame(movie_genres_count)
df_movie_genres_count

,Genero,NumPeliculas
0,Drama,6
1,Acción,6
2,Suspenso,5
3,Ciencia Ficción,4
4,Terror,3
5,Comedia,1


###  **Investigación 3:** ¿Cómo ha afectado la disminución de cineastas a las colaboraciones?
- Objetivo: Investigar si la disminución de cineastas ha afectado la colaboración dentro de la comunidad. Uso de COUNT().

In [47]:
def get_collaborations_count():
  query = """
  MATCH (p:Person)-[:COLLABORATED_WITH]->(colleague)
  RETURN COUNT(DISTINCT p) AS CineastasColaborando
  """
  with driver.session() as session:
    result = session.run(query)
    collaborations_count = result.single()["CineastasColaborando"]
    return collaborations_count

collaborations_count = get_collaborations_count()
print("Colaboraciones entre cineastas:", collaborations_count)

Colaboraciones entre cineastas: 9


>Pista revelada: Las colaboraciones han aumentado, lo que sugiere que la comunidad cinematográfica es más unida y competitiva que antes.

## **Acto 2: Los Vínculos Ocultos**

### **Investigación 4:** Agrupar cineastas por año de unión

- Objetivo: Ver tendencias en la adhesión de nuevos cineastas a lo largo del tiempo.  

In [48]:
def get_cineastas_por_ano():
    query = """
    MATCH (p:Person)
    RETURN p.yearJoined AS AñoIngreso, COLLECT(p.name) AS Cineastas
    ORDER BY AñoIngreso
    """
    with driver.session() as session:
        result = session.run(query)
        cineastas_por_ano = [{"AñoIngreso": record["AñoIngreso"], "Cineastas": record["Cineastas"]} for record in result]
        return cineastas_por_ano

cineastas_por_ano = get_cineastas_por_ano()

df_cineastas_por_ano = pd.DataFrame(cineastas_por_ano)
df_cineastas_por_ano

,AñoIngreso,Cineastas
0,1985,[Roberto Díaz]
1,1990,[Juan Pérez]
2,1998,[Alejandro González]
3,2000,[Carlos Méndez]
4,2003,[Sofia Ramírez]
5,2005,[Laura Fernández]
6,2010,[Carla Mendoza]
7,2012,[Elena Gutiérrez]
8,2015,[Miguel Torres]
9,2018,[Valeria Soto]


### **Investigación 5:** Identificar las películas dirigidas por cada cineasta

- Objetivo: Coleccionar los títulos de películas dirigidas por cada cineasta.  

In [49]:
def get_peliculas_dirigidas():
    query = """
    MATCH (p:Person)-[:DIRECTED]->(m:Movie)
    RETURN p.name AS Director, COLLECT(m.title) AS PeliculasDirigidas
    """
    with driver.session() as session:
        result = session.run(query)
        peliculas_dirigidas = [{"Director": record["Director"], "PeliculasDirigidas": record["PeliculasDirigidas"]} for record in result]
        return peliculas_dirigidas

peliculas_dirigidas = get_peliculas_dirigidas()

df_peliculas_dirigidas = pd.DataFrame(peliculas_dirigidas)
df_peliculas_dirigidas

,Director,PeliculasDirigidas
0,Alejandro González,"[El Último Acto, Destino Inesperado]"
1,Sofia Ramírez,"[Ciudad Oscura, Sueños Rotos]"
2,Juan Pérez,"[Fuego Interno, Horizonte Perdido, Más Allá de..."
3,Carla Mendoza,"[Sin Retorno, El Último Refugio]"
4,Miguel Torres,"[Caos Total, Código Secreto]"
5,Laura Fernández,"[Fantasmas del Pasado, Caminos Cruzados]"
6,Roberto Díaz,"[Guerra de Titanes, El Silencio Mortal, El Fin..."
7,Elena Gutiérrez,"[Dimensión Desconocida, La Verdad Prohibida]"
8,Carlos Méndez,"[El Espejo Roto, Lágrimas en el Viento, Colapso]"


### **Investigación 6:** Agrupar películas por género y director

- Objetivo: Entender cómo los directores han diversificado su producción.  

In [50]:
def get_peliculas_por_genero_y_director():
    query = """
    MATCH (p:Person)-[:DIRECTED]->(m:Movie)-[:BELONGS_TO]->(g:Genre)
    RETURN p.name AS Director, g.name AS Genero, COLLECT(m.title) AS Peliculas
    ORDER BY Director
    """
    with driver.session() as session:
        result = session.run(query)
        peliculas_por_genero_y_director = [{"Director": record["Director"], "Genero": record["Genero"], "Peliculas": record["Peliculas"]} for record in result]
        return peliculas_por_genero_y_director

peliculas_por_genero_y_director = get_peliculas_por_genero_y_director()

df_peliculas_por_genero_y_director = pd.DataFrame(peliculas_por_genero_y_director)
df_peliculas_por_genero_y_director

,Director,Genero,Peliculas
0,Alejandro González,Drama,[El Último Acto]
1,Alejandro González,Acción,[Destino Inesperado]
2,Carla Mendoza,Suspenso,[Sin Retorno]
3,Carla Mendoza,Drama,[El Último Refugio]
4,Carlos Méndez,Terror,[El Espejo Roto]
5,Carlos Méndez,Drama,[Lágrimas en el Viento]
6,Carlos Méndez,Ciencia Ficción,[Colapso]
7,Elena Gutiérrez,Ciencia Ficción,[Dimensión Desconocida]
8,Elena Gutiérrez,Suspenso,[La Verdad Prohibida]
9,Juan Pérez,Acción,[Fuego Interno]


## **Acto 3: Evaluación de la Productividad**

### **Investigación 7:** Contar cuántas películas ha dirigido cada cineasta
- Objetivo: Medir la productividad de cada cineasta.

In [51]:
def get_num_peliculas_dirigidas():
    query = """
    MATCH (p:Person)-[:DIRECTED]->(m:Movie)
    RETURN p.name AS Director, SIZE(COLLECT(m.title)) AS NumPeliculasDirigidas
    ORDER BY NumPeliculasDirigidas DESC
    """
    with driver.session() as session:
        result = session.run(query)
        num_peliculas_dirigidas = [{"Director": record["Director"], "NumPeliculasDirigidas": record["NumPeliculasDirigidas"]} for record in result]
        return num_peliculas_dirigidas

num_peliculas_dirigidas = get_num_peliculas_dirigidas()

df_num_peliculas_dirigidas = pd.DataFrame(num_peliculas_dirigidas)
df_num_peliculas_dirigidas

,Director,NumPeliculasDirigidas
0,Roberto Díaz,4
1,Juan Pérez,3
2,Carlos Méndez,3
3,Alejandro González,2
4,Sofia Ramírez,2
5,Carla Mendoza,2
6,Miguel Torres,2
7,Laura Fernández,2
8,Elena Gutiérrez,2


### **Investigación 8:** Evaluar la red de colaboraciones de cada cineasta
- Objetivo: Medir la cantidad de conexiones de cada cineasta.

In [52]:
def get_num_colaboradores():
    query = """
    MATCH (p:Person)-[:COLLABORATED_WITH]->(other)
    RETURN p.name AS Cineasta, SIZE(COLLECT(other.name)) AS NumColaboradores
    ORDER BY NumColaboradores DESC
    """
    with driver.session() as session:
        result = session.run(query)
        num_colaboradores = [{"Cineasta": record["Cineasta"], "NumColaboradores": record["NumColaboradores"]} for record in result]
        return num_colaboradores

num_colaboradores = get_num_colaboradores()

df_num_colaboradores = pd.DataFrame(num_colaboradores)
df_num_colaboradores

,Cineasta,NumColaboradores
0,Juan Pérez,3
1,Alejandro González,2
2,Carla Mendoza,2
3,Laura Fernández,2
4,Roberto Díaz,2
5,Elena Gutiérrez,2
6,Carlos Méndez,2
7,Valeria Soto,2
8,Miguel Torres,1


### **Investigación 9:** Medir la diversificación de los cineastas en géneros
- Objetivo: Determinar cuántos géneros distintos ha explorado cada cineasta.

In [53]:
def get_num_generos_explorados():
    query = """
    MATCH (p:Person)-[:DIRECTED]->(m:Movie)-[:BELONGS_TO]->(g:Genre)
    RETURN p.name AS Cineasta, SIZE(COLLECT(DISTINCT g.name)) AS NumGenerosExplorados
    ORDER BY NumGenerosExplorados DESC
    """
    with driver.session() as session:
        result = session.run(query)
        num_generos_explorados = [{"Cineasta": record["Cineasta"], "NumGenerosExplorados": record["NumGenerosExplorados"]} for record in result]
        return num_generos_explorados

num_generos_explorados = get_num_generos_explorados()

df_num_generos_explorados = pd.DataFrame(num_generos_explorados)
df_num_generos_explorados

,Cineasta,NumGenerosExplorados
0,Juan Pérez,3
1,Roberto Díaz,3
2,Carlos Méndez,3
3,Alejandro González,2
4,Sofia Ramírez,2
5,Carla Mendoza,2
6,Miguel Torres,2
7,Laura Fernández,2
8,Elena Gutiérrez,2
